In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  4 23:28:32 2022

@author: parkh
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from SIM7 import *

learning_rate = 0.001  
gamma = 1
buffer_limit = 10000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(68,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
        self.number_of_time_list = np.array([1 for x in range(6)])
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        #print(out)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return act
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        return act,act_list
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/FJSP_SIM4.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
            
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
        s = env.reset()
        done = False
        score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        print(a_list)
        print(a)
        s_prime, r, done = env.step(a)
        print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    return Flow_time, machine_util, util, makespan, score
Flow_time, machine_util, util, makespan, score =main()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)
      
    

C:\Users\parkh\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


--------------------------------------------------
flow time: 1639, util : 0.830, makespan : 174
n_episode: 1, score : -120.0, n_buffer : 96, eps : 8.0%
--------------------------------------------------
flow time: 1539, util : 0.836, makespan : 155
n_episode: 2, score : -99.0, n_buffer : 144, eps : 8.0%
--------------------------------------------------
flow time: 1539, util : 0.849, makespan : 177
n_episode: 3, score : -96.0, n_buffer : 192, eps : 8.0%
--------------------------------------------------
flow time: 1560, util : 0.822, makespan : 157
n_episode: 4, score : -108.0, n_buffer : 240, eps : 8.0%
--------------------------------------------------
flow time: 1587, util : 0.818, makespan : 167
n_episode: 5, score : -112.0, n_buffer : 288, eps : 8.0%
--------------------------------------------------
flow time: 1525, util : 0.833, makespan : 158
n_episode: 6, score : -103.0, n_buffer : 336, eps : 7.9%
--------------------------------------------------
flow time: 1501, util : 0.84

<ipython-input-2-fc3851ac1087>:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 1591, util : 0.815, makespan : 163
n_episode: 20, score : -115.0, n_buffer : 1008, eps : 7.8%
--------------------------------------------------
flow time: 1822, util : 0.907, makespan : 189
n_episode: 21, score : -67.0, n_buffer : 1056, eps : 7.8%
--------------------------------------------------
flow time: 1815, util : 0.861, makespan : 200
n_episode: 22, score : -203.0, n_buffer : 1104, eps : 7.8%
--------------------------------------------------
flow time: 1516, util : 0.917, makespan : 183
n_episode: 23, score : -74.0, n_buffer : 1152, eps : 7.8%
--------------------------------------------------
flow time: 1376, util : 0.867, makespan : 167
n_episode: 24, score : -90.0, n_buffer : 1200, eps : 7.8%
--------------------------------------------------
flow time: 1373, util : 0.885, makespan : 173
n_episode: 25, score : -81.0, n_buffer : 1248, eps : 7.8%
--------------------------------------------------
flow time: 1511, 

--------------------------------------------------
flow time: 1420, util : 0.899, makespan : 180
n_episode: 73, score : -70.0, n_buffer : 3552, eps : 7.3%
--------------------------------------------------
flow time: 1369, util : 0.892, makespan : 199
n_episode: 74, score : -75.0, n_buffer : 3600, eps : 7.3%
--------------------------------------------------
flow time: 1329, util : 0.901, makespan : 174
n_episode: 75, score : -73.0, n_buffer : 3648, eps : 7.3%
--------------------------------------------------
flow time: 1507, util : 0.912, makespan : 185
n_episode: 76, score : -65.0, n_buffer : 3696, eps : 7.2%
--------------------------------------------------
flow time: 1464, util : 0.908, makespan : 191
n_episode: 77, score : -65.0, n_buffer : 3744, eps : 7.2%
--------------------------------------------------
flow time: 1225, util : 0.897, makespan : 176
n_episode: 78, score : -66.0, n_buffer : 3792, eps : 7.2%
--------------------------------------------------
flow time: 1470, ut

...

--------------------------------------------------
flow time: 1178, util : 0.879, makespan : 190
n_episode: 762, score : -81.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1189, util : 0.929, makespan : 162
n_episode: 763, score : -44.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1159, util : 0.923, makespan : 164
n_episode: 764, score : -49.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1157, util : 0.882, makespan : 180
n_episode: 765, score : -79.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1178, util : 0.879, makespan : 190
n_episode: 766, score : -81.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1242, util : 0.912, makespan : 186
n_episode: 767, score : -58.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1159, util : 0.923, makespan : 164
n_episode: 815, score : -49.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1291, util : 0.914, makespan : 168
n_episode: 816, score : -53.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1159, util : 0.923, makespan : 164
n_episode: 817, score : -49.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1494, util : 0.870, makespan : 196
n_episode: 818, score : -93.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1266, util : 0.932, makespan : 172
n_episode: 819, score : -44.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1221, util : 0.933, makespan : 160
n_episode: 820, score : -41.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1415, util : 0.887, makespan : 194
n_episode: 868, score : -108.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1206, util : 0.902, makespan : 174
n_episode: 869, score : -62.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1437, util : 0.906, makespan : 187
n_episode: 870, score : -64.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1258, util : 0.896, makespan : 178
n_episode: 871, score : -71.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1152, util : 0.902, makespan : 171
n_episode: 872, score : -62.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1168, util : 0.892, makespan : 191
n_episode: 873, score : -84.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow t

--------------------------------------------------
flow time: 1125, util : 0.934, makespan : 160
n_episode: 921, score : -50.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1122, util : 0.891, makespan : 180
n_episode: 922, score : -77.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1172, util : 0.916, makespan : 166
n_episode: 923, score : -54.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1172, util : 0.916, makespan : 166
n_episode: 924, score : -54.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1172, util : 0.916, makespan : 166
n_episode: 925, score : -54.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1180, util : 0.884, makespan : 174
n_episode: 926, score : -92.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 1134, util : 0.929, makespan : 178
n_episode: 974, score : -44.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1144, util : 0.910, makespan : 173
n_episode: 975, score : -59.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1262, util : 0.877, makespan : 168
n_episode: 976, score : -78.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1122, util : 0.885, makespan : 163
n_episode: 977, score : -69.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1157, util : 0.932, makespan : 160
n_episode: 978, score : -42.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 1157, util : 0.882, makespan : 180
n_episode: 979, score : -79.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow ti